In [2]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import requests
import re
import time
from tqdm import tqdm
import json
from datetime import datetime, timedelta, timezone

In [ ]:
with open("../data/鉄道人身事故データベース.html", "r") as f:
    html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.findAll("table")[0].findAll("tr")
    with open("../data/jinshin.csv", "w", newline="", encoding="utf-8") as file:
        csv_write = csv.writer(file)
        for tr in a:
            data = []
            for cell in tr.findAll(["td", "th"]):
                if (not "詳細" in cell.get_text()) and cell.get_text() != "":
                    data.append(cell.get_text())
            if len(data) == 3:
                data.append("")
            for url in tr.findAll("a"):
                data.append(url.get("href"))
            csv_write.writerow(data)

In [ ]:
df = pd.read_csv("../data/jinshin.csv", header=0, names=["date", "line", "place", "injury", "url"])
data = []
sentences = []
for url in tqdm(df["url"]):
    d = BeautifulSoup(requests.get(url).text,'html.parser')
    a = d.findAll("a",text=re.compile("魚拓"))
    if a :
        data.append(a[0].get("href"))
    else :
        data.append("")
    sentence = d.findAll("p",class_="card-text")
    if sentence:
        sentences.append(sentence[0].get_text().replace(" ","").replace("\n","").replace("　",""))
    else :
        sentences.append("")
    time.sleep(0.01)
# 保存
save_data = np.array(data)
save_sentences = np.array(sentences)
np.save("archive_url3",save_data)
np.save("news_sentences", save_sentences)
# 付け加えて保存
df["archive"] = data
df["sentence"] = sentences
df.to_csv("../data/jinshin_archive.csv")

In [ ]:
url = df["url"][0]
d = BeautifulSoup(requests.get(url).text, 'html.parser')

gyotaku = d.findAll("a", text =re.compile("魚拓"))[0].get("href")
print(gyotaku)
print(d.findAll('p',class_="card-text")[0].get_text())

In [9]:
df = pd.read_csv("../data/jinshin_archive.csv", header=0, names=["date", "line", "place", "injury", "url", "archive", "sentence"])
data = []
sentences = []
for url in tqdm(df["url"]):
    d = BeautifulSoup(requests.get(url).text,'html.parser')
    sentence = d.findAll("p",class_="card-text")
    if sentence:
        sentences.append(sentence[0].get_text().replace(" ","").replace("\n","").replace("　",""))
    else :
        sentences.append("")
    time.sleep(0.01)
# 保存
save_sentences = np.array(sentences)
np.save("news_sentences", save_sentences)
# 付け加えて保存
df["sentence"] = sentences
df.to_csv("../data/jinshin_archive_sentence.csv")

100%|██████████| 14897/14897 [1:01:16<00:00,  4.05it/s]


In [29]:
with open("../data/JR東/東日本旅客鉄道東海道線.geojson") as f:
    df = json.load(f)
print(df["features"][0]["properties"]["name"])

東京


In [10]:
df = pd.read_csv("../data/jinshin_archive_sentence.csv",index_col=0)
df.date = df.date.apply(lambda x: datetime.strptime(x, '%Y/%m/%d %H:%M').time())
df.sort_values("date", ascending=True, inplace=True)
df.date = df.date.apply(lambda x: x.strftime("%H:%M"))
a = df["date"].to_numpy()
list = {}
for i in a:
    if i in list:
        list[i] += 1
    else:
        list[i] = 1
with open("timeonly.json", "w") as f:
    jsonhoge = json.dump(list, f, indent=2)


In [3]:
with open("./timeonly.json","r") as f:
    a = json.load(f)
    for hour in range(24):
        for minute in range(60):
            key = str(hour).zfill(2) + ":" + str(minute).zfill(2)
            if  not key in a:
                a[key] = 0
with open("timeonly.json","w") as f:
    json.dump(a,f, indent=2)

In [14]:
section = [0, 10, 20, 30, 40, 50, 60]
res = []
with open("./timeonly.json", "r") as f:
    d = json.load(f)
    for hour in range(24):
        for i in range(len(section)-1):
            start = section[i]
            end = section[i+1]
            dataarray = []
            for minute in range(start, end):
                key = str(hour).zfill(2) + ":" + str(minute).zfill(2)
                dataarray.append(d[key])
            tmp = np.array(dataarray)
            c_array = np.percentile(tmp,[0,25,50,75,100])
            res.append({"hour": hour, "minute": section[i], "min": c_array[0], "quarter":c_array[1], "median": c_array[2], "quarter3": c_array[3], "max": c_array[4]})
# print(res)
with open("jinshin4.json","w") as f:
    json.dump(res,f,indent=2)